## Import packages

In [ ]:
import ck.kernel as ck
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']
print "pandas: v%s" % pd.version.version
print "NumPy: v%s" % np.version.version
print "Matplotlib: v%s" % matplotlib.__version__
print "JSON: v%s" % json.__version__

In [ ]:
%matplotlib inline

## Find results

In [ ]:
dataset = 'SGEMM_NT'
data_uoa =  dataset + '-explore-f-n'
module_uoa = 'experiment'

In [ ]:
r=ck.access({'action':'list_points', 'module_uoa':module_uoa, 'data_uoa':data_uoa})
if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

## Show results

In [ ]:
data_list  = []
index_list = []

for point in r['points']:
    with open(os.path.join(r['path'], 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    # Data.    
    GFLOPS = point_data.get("##characteristics#run#run_time_state#EXECUTION#Gflops/s#all") # GFLOPS ~ Gflops/s
    GPU    = point_data.get("##characteristics#run#run_time_state#file_2_energy#all") # Mali-T628 GPU
    memory = point_data.get("##characteristics#run#run_time_state#file_1_energy#all") # memory
    big    = point_data.get("##characteristics#run#run_time_state#file_0_energy#all") # Cortex-A15 CPU
    LITTLE = point_data.get("##characteristics#run#run_time_state#file_3_energy#all") # Cortex-A7 CPU
    data_list.append(GFLOPS + GPU + memory + big  + LITTLE)
    
    # Row index.
    cl_file = point_data.get("##characteristics#run#run_time_state#METADATA#file#all_unique")[0]
    lws_j = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_j#all_unique")[0]
    lws_i = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_i#all_unique")[0]
    local_work_size = ('(%s, %s)' % (lws_j, lws_i))
    matrix_order = point_data.get("##characteristics#run#run_time_state#CMD_LINE_ARGS#matrix_order#all_unique")[0]
    index_list.append((cl_file, local_work_size, matrix_order))

In [ ]:
num_repetitions = 4 # TODO: get from points
metrics = ['GPU, Gflops/s', 'GPU, Joules', 'Memory, Joules', 'big CPU, Joules', 'LITTLE CPU, Joules']
repetitions_array = range(num_repetitions) * len(metrics)
metrics_array = [metric for metric in metrics for n in range(num_repetitions)]
cols_mi = pd.MultiIndex.from_arrays(arrays=[metrics_array, repetitions_array], names=['Metric', 'Repetition'])
rows_mi = pd.MultiIndex.from_tuples(names=['OpenCL program', 'Local work size', 'Matrix order'], tuples=index_list)
df = pd.DataFrame(data=data_list, index=rows_mi, columns=cols_mi).sortlevel()
df.index = df.index.droplevel('Local work size') # not interested in here as it's fixed

In [ ]:
df \
    .unstack('OpenCL program') \
    .stack(level='Repetition')

### Performance with errors

In [ ]:
mean = df['GPU, Gflops/s'].mean(axis=1)
std = df['GPU, Gflops/s'].std(axis=1)
ymax = np.int64(mean.max() + std.max())

plot = mean \
    .unstack(level='OpenCL program') \
    .plot(yerr=std.unstack(level='OpenCL program'),
        title='Gflops/s vs Matrix order',
        kind='bar', figsize=(12,8), colormap=matplotlib.cm.autumn,
        ylim=(0, ymax), yticks=range(ymax))

### All performance and energy means

In [ ]:
df_mean_all = df \
    .unstack('OpenCL program') \
    .stack(level='Repetition') \
    .groupby(level='Matrix order') \
    .mean()
df_mean_all

### All energy means

In [ ]:
df_mean_energy_all = df[['GPU, Joules', 'Memory, Joules', 'big CPU, Joules', 'LITTLE CPU, Joules']] \
    .unstack('OpenCL program') \
    .stack(level='Repetition') \
    .groupby(level='Matrix order') \
    .mean()
df_mean_energy_all

In [ ]:
df_mean_energy_all \
    .stack('OpenCL program') \
    .loc[[896, 1024]] \
    .plot(kind='bar', figsize=(12,8), colormap=matplotlib.cm.autumn, stacked=True)

### GPU and memory energy means

In [ ]:
df_mean_energy_gpu_mem = df[['GPU, Joules', 'Memory, Joules']] \
    .unstack('OpenCL program') \
    .stack(level='Repetition') \
    .groupby(level='Matrix order') \
    .mean()
df_mean_energy_gpu_mem

In [ ]:
df_mean_energy_gpu_mem = df_mean_energy_gpu_mem \
    .swaplevel(0, 1, axis=1) \
    .sortlevel(0, axis=1)
df_mean_energy_gpu_mem

In [ ]:
df_mean_energy_gpu_mem_plot = df_mean_energy_gpu_mem \
    .stack('OpenCL program') \
    .loc[[896, 1024]] \
    .plot(kind='bar', figsize=(12,12), colormap=matplotlib.cm.autumn, stacked=True)
df_mean_energy_gpu_mem_plot

#### Dump to files

In [ ]:
with open('%s-energy-gpu-mem_tmp.tex' % data_uoa, 'w') as tex_file:
    tex_file.write(df_mean_energy_gpu_mem.to_latex())

In [ ]:
df_mean_energy_gpu_mem_plot.get_figure().savefig('%s-energy-gpu-mem_tmp.pdf' % data_uoa)